In [1]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import spacy

Le but de ce notebook est de définir 3 grande techniques pour extraire des relations en s'appuyant sur trois grandes bases grammatical et les graphs de dependency de spacy

In [2]:
rootdir=os.getcwd()+'/dataset/' 
directories=[]
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        directories.append(d)

In [3]:
datas={}
nlp = spacy.load("en_core_web_sm")

for i in directories:
    pref=str(i.split('/')[-1])
    files = [f for f in listdir(i) if isfile(join(i, f))]
    for f in files:
        if f[-4:]=='.txt':
            suf=f[:-4]
            name=pref+suf
            with open(i+'/'+f, "r") as file:
                data = file.read().replace("\n", "")
            article = data.replace(u"\xa0", u" ")
            datas[name]=nlp(article)
        
# saves doc object in datas dictionary for all articles 

In [4]:
text_names = [f for f in datas.keys()]

list of dependency labels
https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md

https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf (descriptions)

https://suttipong-kull.medium.com/how-to-extract-subject-verb-and-object-by-nlp-4149323a7d7d


# Note: All extractions work better with Coreference resolution before

# SVO construction

Simple sentences subject-verbe-object. It aims to extract two-entity tuples, given a selected set of verbs 

!!! note ORG are recognize if starting with capital letter

In [5]:
test3=nlp("Facebook bought instagram and whatsapp")
for s in test3.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

  Facebook   NNP      nsubj     bought PROPN
    bought   VBD       ROOT     bought VERB
 instagram    NN       dobj     bought NOUN
       and    CC         cc  instagram CCONJ
  whatsapp    NN       conj  instagram NOUN


In [6]:
# passive is detected with nsubjpass and auxpass
test=nlp("Twitter was bought by facebook")
for s in test.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

   Twitter   NNP  nsubjpass     bought PROPN
       was   VBD    auxpass     bought AUX
    bought   VBN       ROOT     bought VERB
        by    IN      agent     bought ADP
  facebook   NNP       pobj         by PROPN


In [7]:
test3=nlp("Instagram and its owner Facebook compete with tiktok")
for s in test3.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

 Instagram   NNP      nsubj    compete PROPN
       and    CC         cc  Instagram CCONJ
       its  PRP$       poss      owner PRON
     owner    NN       conj  Instagram NOUN
  Facebook   NNP      appos      owner PROPN
   compete    VB       ROOT    compete VERB
      with    IN       prep    compete ADP
    tiktok    NN       pobj       with NOUN


In [8]:
for s in test.sents:
    print(s.ents)

[Twitter]


In [9]:
for s in test.sents:
    print("sentence entities types")
    for t in s.ents:
        print(t.label_)

sentence entities types
PRODUCT


Cela montre que spacy mispecify souvent les "organisations". certaines entitées ne sortent pas également (comme Facebook sur la deuxième phrase). Les entités en minuscules ne sont aussi pas relevées. Celà suggère qu'il faut peut être parser les entitées à partir d'une base de données d'entreprises. Note: Compute entities at the article level scales down the problem

In [10]:
def list_entities(text):
    ents_=text.ents
    ents=[]
    for x in ents_:
        if x.text not in ents:
            if x.label_=="ORG" or x.label_=="PRODUCT" or x.label_=="GPE":
                ents.append(x.text)
    return ents


def find_org(s, ents):
    i=0
    ind=[]
    for token in s:
        if str(token) in ents: 
            ind.append(i)
        i=i+1
    return ind

def check_synonym_SVO(s, list_):
    for token in s:
        if token.dep_=="ROOT":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False      

def find_sbj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubj":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects

def find_nsubjpass(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubjpass":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubjpass": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects
            
def find_obj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="dobj" or s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="dobj" or s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_attr(s, list_):
    for token in s:
        if token.dep_=="attr":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False
            
def find_obj_attr(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_acl(s, list_):
    for token in s:
        if token.dep_=="acl":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False  

def find_obj_acl(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects


In [11]:
# assume we take all root verbs
tuples=[]
relation="is_owner"
synonyms=["buy","acquire","purchase","own"]
test=nlp("Twitter was bought by Facebook. Facebook owns Instagram and Whatsapp. Instagram and its owner Facebook want to buy Tiktok. Apple competes with Microsoft. John bought Michelle a candy.")
ents=list_entities(test)
for s in test.sents:
    dict_={}
    org=find_org(s,ents)
    print(org)
    if len(org)>=2:
        SVO=check_synonym_SVO(s,synonyms)
        if SVO==True:
            sbj=find_sbj(s,org)
            if len(sbj)>0: 
                obj=find_obj(s,org)
                if len(obj)>0:
                    dict_['relation']=relation
                    ## add ID later dict_['relation']
                    dict_['sentence']=s
                    dict_['entities_A']=sbj
                    dict_['entities_B']=obj
                    tuples.append(dict_)
            else:
                nsubjpass=find_nsubjpass(s,org)
                if len(nsubjpass)>0:
                    obj=find_obj(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=obj
                        dict_['entities_B']=sbj
                        tuples.append(dict_)
                else:
                    continue
        else:
            continue
    else:
        continue
        

[0, 4]
[0, 2, 4]
[0, 4, 8]
[3]
[]


In [12]:
tuples

[{'relation': 'is_owner',
  'sentence': Twitter was bought by Facebook.,
  'entities_A': [Facebook],
  'entities_B': []},
 {'relation': 'is_owner',
  'sentence': Facebook owns Instagram and Whatsapp.,
  'entities_A': [Facebook],
  'entities_B': [Instagram, Whatsapp]}]

# attributes kind of patterns

This pattern allow us to filter via the "attr", we select a list of words that shall correlate with the relation in question and filter the entities

In [13]:
test3=nlp("Facebook and Instagram are competitors of Tiktok and Twitter")
for s in test3.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

  Facebook   NNP      nsubj        are PROPN
       and    CC         cc   Facebook CCONJ
 Instagram   NNP       conj   Facebook PROPN
       are   VBP       ROOT        are AUX
competitors   NNS       attr        are NOUN
        of    IN       prep competitors ADP
    Tiktok   NNP       pobj         of PROPN
       and    CC         cc     Tiktok CCONJ
   Twitter   NNP       conj     Tiktok PROPN


In [14]:
test3=nlp("Engie is a supplier of many businesses")
for s in test3.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

     Engie    NN      nsubj         is NOUN
        is   VBZ       ROOT         is AUX
         a    DT        det   supplier DET
  supplier    NN       attr         is NOUN
        of    IN       prep   supplier ADP
      many    JJ       amod businesses ADJ
businesses   NNS       pobj         of NOUN


In [15]:
test3=nlp("Facebook is a company from Meta group")
for s in test3.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

  Facebook   NNP      nsubj         is PROPN
        is   VBZ       ROOT         is AUX
         a    DT        det    company DET
   company    NN       attr         is NOUN
      from    IN       prep    company ADP
      Meta   NNP   compound      group PROPN
     group    NN       pobj       from NOUN


In [16]:
tuples=[]
relation="is_owner"  # note: adapt for one way relationships
synonyms=["company","firm","branch"]
test=nlp("Twitter was bought by Facebook. Engie is a supplier of RATP. Facebook is a company from Meta group. Engie is a firm that competes with EDF.")
ents=list_entities(test)
for s in test.sents:
    dict_={}
    org=find_org(s,ents)
    print(org)
    if len(org)>=2:
        attr=check_synonym_attr(s,synonyms)
        if attr==True:
            sbj=find_sbj(s,org)
            if len(sbj)>0: 
                obj=find_obj_attr(s,org)
                if len(obj)>0:
                    dict_['relation']=relation
                    ## add ID later dict_['relation']
                    dict_['sentence']=s
                    dict_['entities_A']=sbj
                    dict_['entities_B']=obj
                    tuples.append(dict_)

[0, 4]
[0]
[0, 5]
[0, 7]


In [17]:
tuples

[{'relation': 'is_owner',
  'sentence': Facebook is a company from Meta group.,
  'entities_A': [Facebook],
  'entities_B': [Meta]},
 {'relation': 'is_owner',
  'sentence': Engie is a firm that competes with EDF.,
  'entities_A': [Engie],
  'entities_B': [EDF]}]

# ACL based

A clausal modifier of noun (acl) is either an infinitive clause, a participial clause, or a clausal complement that modifies the head of a noun phrase.
Ex:

In [18]:
test=nlp("Instagram was the company acquired by Facebook")
for s in test.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

 Instagram   NNP      nsubj        was PROPN
       was   VBD       ROOT        was AUX
       the    DT        det    company DET
   company    NN       attr        was NOUN
  acquired   VBN        acl    company VERB
        by    IN      agent   acquired ADP
  Facebook   NNP       pobj         by PROPN


In [19]:
test=nlp("Instagram, the company bought by Facebook, is a social media.")
for s in test.sents:
    for token in s:
        print('%10s %5s %10s %10s %s'%(token.text, token.tag_, token.dep_, token.head, token.pos_))

 Instagram   NNP      nsubj         is PROPN
         ,     ,      punct  Instagram PUNCT
       the    DT        det    company DET
   company    NN      appos  Instagram NOUN
    bought   VBN        acl    company VERB
        by    IN      agent     bought ADP
  Facebook   NNP       pobj         by PROPN
         ,     ,      punct  Instagram PUNCT
        is   VBZ       ROOT         is AUX
         a    DT        det      media DET
    social    JJ       amod      media ADJ
     media   NNS       attr         is NOUN
         .     .      punct         is PUNCT


In [20]:
tuples=[]
relation="is_owner"  # note: adapt for one way relationships
synonyms=["buy","acquire","purchase","own"]
test=nlp("Twitter was bought by Facebook. Instagram, the company bought by Facebook, is a social media. Facebook is a company from Meta group. Instagram was the company acquired by Facebook.")
ents=list_entities(test)
for s in test.sents:
    print(s)
    dict_={}
    org=find_org(s,ents)
    print(org)
    if len(org)>=2:
        acl=check_synonym_acl(s,synonyms)
        if acl==True:
            sbj=find_sbj(s,org)
            if len(sbj)>0: 
                obj=find_obj_acl(s,org)
                if len(obj)>0:
                    dict_['relation']=relation
                    ## add ID later dict_['relation']
                    dict_['sentence']=s
                    dict_['entities_A']=sbj
                    dict_['entities_B']=obj
                    tuples.append(dict_)

Twitter was bought by Facebook.
[0, 4]
Instagram, the company bought by Facebook, is a social media.
[6]
Facebook is a company from Meta group.
[0, 5]
Instagram was the company acquired by Facebook.
[6]


In [21]:
tuples

[]

Tuples empty because spacy does not recognize Instagram as entity